In [3]:
from datetime import datetime
import pandas as pd
import numpy as np
import requests
import sqlite3
from bs4 import BeautifulSoup

# Log each step of the ETL process to a text file
def log_progress(message):
    time_stamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    log_entry = f"{time_stamp} : {message}\n"
    with open('code_log.txt', 'a') as log_file:
        log_file.write(log_entry)

# Extract: Scrape a Wikipedia page and parse the target table using BeautifulSoup
def extract(url, table_attribs):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    table = soup.find('table', attrs=table_attribs)
    if table is None:
        print("Target table with specified class not found.")
        return pd.DataFrame()

    # Parse table headers
    headers = [th.get_text(strip=True) for th in table.find_all('tr')[0].find_all('th')]

    # Parse table rows
    rows = []
    for tr in table.find_all('tr')[1:]:
        cols = tr.find_all(['td', 'th'])
        row = [col.get_text(strip=True).replace(',', '') for col in cols]
        if len(row) == len(headers):
            rows.append(row)

    df = pd.DataFrame(rows, columns=headers)

    # Rename and convert the 'Market cap' column
    col_name = next((c for c in df.columns if 'Market cap' in c and 'US$' in c), None)
    if col_name:
        df = df.rename(columns={col_name: 'MC_USD_Billion'})
        df['MC_USD_Billion'] = pd.to_numeric(df['MC_USD_Billion'], errors='coerce')
    else:
        print("Column 'Market cap (US$ billion)' not found.")

    return df

# Transform: Convert market cap values from USD to GBP, EUR, and INR using exchange rates
def transform(df, csv_path):
    exchange_rate_df = pd.read_csv(csv_path)
    exchange_rate = exchange_rate_df.set_index('Currency').to_dict()['Rate']
    df['MC_GBP_Billion'] = np.round(df['MC_USD_Billion'] * exchange_rate['GBP'], 2)
    df['MC_EUR_Billion'] = np.round(df['MC_USD_Billion'] * exchange_rate['EUR'], 2)
    df['MC_INR_Billion'] = np.round(df['MC_USD_Billion'] * exchange_rate['INR'], 2)
    return df

# Load: Export the transformed DataFrame to a CSV file
def load_to_csv(df, output_path):
    df.to_csv(output_path, index=False)
    print(f"Dataframe successfully saved to {output_path}")
    log_progress(f"Dataframe saved to {output_path}")

# Load: Store the transformed DataFrame into an SQLite database
def load_to_db(df, sql_connection, table_name):
    try:
        df.to_sql(table_name, sql_connection, if_exists='replace', index=False)
        print("Data loaded into SQLite database.")
        log_progress("Data loaded into SQLite database.")
    except Exception as e:
        print(f"An error occurred: {e}")
        log_progress(f"Error while loading into DB: {e}")

# Query: Execute a query against the SQLite database
def run_query(query_statement, sql_connection):
    try:
        cursor = sql_connection.cursor()
        cursor.execute(query_statement)
        results = cursor.fetchall()
        print(f"Query: {query_statement}")
        for row in results:
            print(row)
        log_progress(f"Executed query: {query_statement}")
    except Exception as e:
        print(f"An error occurred: {e}")
        log_progress(f"Query failed: {e}")

# ===== MAIN ETL EXECUTION =====
log_progress("ETL process started.")

url = "https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks"
table_attribs = {'class': 'wikitable'}
df = extract(url, table_attribs)
log_progress("Data extraction completed.")

csv_path = 'exchange_rate.csv'
df_transformed = transform(df, csv_path)
log_progress("Data transformation completed.")

output_path = 'transformed_data.csv'
load_to_csv(df_transformed, output_path)

connection = sqlite3.connect('Banks.db')
load_to_db(df_transformed, connection, 'Largest_banks')

# Run a Query test
run_query("SELECT * FROM Largest_banks LIMIT 5;", connection)
connection.close()

log_progress("ETL process completed.")

Dataframe successfully saved to transformed_data.csv
Data loaded into SQLite database.
